# Link2Feed Client Mapping Project: Analyzing client population to find new service opportunities

## Notebook 1 of 2: Data Cleaning / Wrangling Notebook

### About the Data Set: 
This data set is pulled from the Link2Feed client intake software system that is implemented by the Food Bank of Central and Eastern NC (FB of CENC). Whenever a client is enrolled in CFSP or TEFAP programs they enter their information into the Link2Feed system so the FB of CENC and associated food pantries can track usage and ensure that persons are qualified to receive the services from the FB that they are requesting. The data set is over 2.5 GB, covers a time frame from October 2019 to present, has almost 200k rows and over 480 columns of data. Most of the data is demographic information. 

From this data set, the FB of CENC wants to visualize where their clients live. These clients are mostly senior citizens, who rely on governmental assistance of some sort, may or may not have difficulty with transportation, have very low income, and are more disadvantaged than average citizens. Because this population is disadvantaged, it is in both the client's and the Food Bank's best interest to ensure food is being distributed in the most efficient way possible. The goal of this project to develop an interactive dashboard that allows the Link2Feed stakeholders visualize where clients live in relation to operating food pantries to determine where new services need to be added. By mapping out clusters of clients geographically, and associated food pantries and distribution centers, we may be able to identify more strategic locations for food distributors which can result in more efficient services to a disadvantaged population. 

### Hypothesis: 
Food Bank CFSP and TEFAP client populations will receive services in a way that saves them time, and transportation costs when new food pantries are opened in locations closer to where they live that provide greater accessibility than the current pantry distribution. 

### Methods: 
The project will be conducted using python, pandas, geopandas, geopy, folium, dash, and Heroku. The data sources will be client information from the link2feed system provided by the Food Bank, county shapefile information made publicly available by the state of NC, and food pantry location data provided by the Food Bank. These three sources will be integrated into an interactive map that will be distributed to the Food Bank as a tool to aid in decision making. Finally, this project will be broken down into two separate notebooks. Notebook 1 is for importing, cleaning and combining data to prepare for analysis. Notebook two is designed for map creation and will be used as a presentation tool. 

### Notes: 
The notebook is broken down into three separate parts. Part 1 is data import. Part 2 is link2Feed client data cleaning. Part 3 is agency data set cleaning, geocoding, and combining three separate datasets. Code blocks are grouped to contain related code. 

## Part 1: Link2Feed Data Import

In [1]:
#import all necessary libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import re

In [2]:
clientMap = pd.read_csv(r'C:\Users\htwal\Jupyter Projects\6a.food_bank_client_mapping\Raw Data\clientMaster.csv')

The original data set was very large (over 2.5 GB and almost 200k rows of data with 483 columns). I identified the data I wanted for this analysis and created a program in pycharm to open the original file and extract the specific information I wanted. The code above imports the data set specific to mapping data. This keeps run times lower and this notebook faster. 

In [3]:
clientMap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195735 entries, 0 to 195734
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          195735 non-null  int64  
 1   Visit Date          195735 non-null  object 
 2   Client ID           195702 non-null  float64
 3   Client Status       195702 non-null  object 
 4   Latitude            171751 non-null  float64
 5   Longitude           171751 non-null  float64
 6   Client CSFP Status  157559 non-null  object 
 7   TEFAP  Eligibility  42127 non-null   object 
 8   Program Name        195735 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 13.4+ MB


In [4]:
clientMap['Program Name'].value_counts()

CSFP Visit            117658
TEFAP Pantry Visit     42127
Food Pantry Visit      35950
Name: Program Name, dtype: int64

In [5]:
clientMap['Client Status'].value_counts()

active      195697
deceased         5
Name: Client Status, dtype: int64

In [6]:
clientMap['Client CSFP Status'].value_counts()

active               118125
closed                15886
waiting-list          13524
deceased               4174
suspended              4026
denied                 1092
inactive, opt-out       334
inactive, moved         284
inactive, unknown        54
deceased, unknown        52
deceased, moved           5
deceased, opt-out         2
denied, unknown           1
Name: Client CSFP Status, dtype: int64

**There is a discrepancy between the 'Client Status' column and the 'Client CSFP Status' column.** Client status only shows 5 deceased persons where as Client CSFP Status shows 4174 deceased!

Since the purpose is to map where each client lives in relation to all the food pantries available, we will need recent clients who are active, their client ID, their location, and their preferred agency (which is the food pantry they use). For the initial dataframe, the following columns will be used: client data, client ID, Client Status, latitude and longitude.

In [7]:
#list of column labels for the new DF
clientMap = clientMap.drop(['Unnamed: 0', 'Client Status'], axis=1)

#create the new DF as a copy from the original
clientMap.head()

,Visit Date,Client ID,Latitude,Longitude,Client CSFP Status,TEFAP Eligibility,Program Name
0,2019-08-08,3331765.0,36.241660,-77.804230,active,NaN,CSFP Visit
1,2019-08-26,3104707.0,35.866110,-86.361570,deceased,NaN,CSFP Visit
2,2019-09-12,4365596.0,35.818421,-78.314352,closed,NaN,CSFP Visit
3,2019-09-24,3228908.0,36.178890,-77.671480,active,NaN,CSFP Visit
4,2019-10-08,3364785.0,36.326834,-77.586209,closed,NaN,CSFP Visit


At this point we have the dataframe we want. Going forward the dataframe used for cleaning, wrangling and analysis will be `clientMap`

## Part 2: Link2Feed Data Cleaning

In [8]:
#change the date column type to datetime
clientMap['Visit Date'] = pd.to_datetime(clientMap['Visit Date'])

#there are many IDs that have duplicates so we will delete the duplicates and keep only the most recent
clientMap.drop_duplicates(subset='Client ID', keep='last', inplace=True)

#now filter out all members who are not 'active' in the `client status` column
clientMap = clientMap[
    ((clientMap['Client CSFP Status'] == 'active') 
    | (clientMap['Client CSFP Status'] == 'closed') 
    | (clientMap['Client CSFP Status'] == 'waiting-list') 
    | (clientMap['Client CSFP Status'] == 'suspended'))
]

#remove all rows with null information in the lat/lon and client ID columns
checkCols = ['Client ID', 'Latitude', 'Longitude']
clientMap = clientMap.dropna(subset=checkCols)

#now that all NaN in `Client ID` is removed, change `Client ID` to int
clientMap['Client ID'] = clientMap['Client ID'].astype(int)

#save for use in next notebook
clientMap.to_csv(r'C:\Users\htwal\Jupyter Projects\6a.food_bank_client_mapping\Processed Data\ClientMapping.csv')

In [9]:
#snapshot of final cleaned data set
clientMap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8087 entries, 283 to 195734
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Visit Date          8087 non-null   datetime64[ns]
 1   Client ID           8087 non-null   int32         
 2   Latitude            8087 non-null   float64       
 3   Longitude           8087 non-null   float64       
 4   Client CSFP Status  8087 non-null   object        
 5   TEFAP  Eligibility  862 non-null    object        
 6   Program Name        8087 non-null   object        
dtypes: datetime64[ns](1), float64(2), int32(1), object(3)
memory usage: 473.8+ KB


In [10]:
clientMap['Client CSFP Status'].value_counts()

active          5254
closed          1767
waiting-list     778
suspended        288
Name: Client CSFP Status, dtype: int64

### Conclusion for Link2Feed client data cleaning:
Overall the dataset was cleaned by dropping unnecessary columns. Then duplicate IDs were removed. I kept the most recent unique ID because that should be the most up to date information for that particular client. Next, I filtered out all clients that were not listed as active, this ensures we only have members that are enrolled, alive, and participating in the CFSP program. Finally I dropped all rows with NaN values in Lat/Lon and ID columns. In total we are left with unique, active clients. 

## Part 3: Pantry Location Data

I was given three total data sets for agency data, all of which had different data and agencies on them. Data set 1 was produced by a database query and contains all agencies, active and inactive over the history of the data set. Data set 2 is a file of the current distribution schedule for the Food Bank, so it contains all the agencies that are currently being delivered to. Data set 3 is active agencies that are provided by the Link2Feed system. I will use data set 1 as the master data set since it is the largest and has the most overlap between all three sets, and I will filter out all inactive agencies (done in a separate IDE and saved, work not shown in this notebook). I will use the other two data sets to fill in missing data in the master to ensure locations are accurate. 

The nature of the data sets makes combining them into one difficult. There could be agencies that appear in one, two, or alll three data sets. They may have different addresses or locations for the same agency in different data sets. I chose the master data set to be the default address, unless its address is Nan. If the location in the master data frame is NaN, or the agency did not appear in the master data frame, I backfilled the agency location information from the other data frames into the master. After all backfilling was complete the produce was one master uniform data frame that is ready for mapping. 

In [11]:
#import all three data sets:
masterAgency = pd.read_csv(r"C:\Users\htwal\Jupyter Projects\6a.food_bank_client_mapping\Raw Data\agency1query.csv")
distributionAgency = pd.read_csv(r"C:\Users\htwal\Jupyter Projects\6a.food_bank_client_mapping\Raw Data\agency2dist.csv")
l2fAgency = pd.read_csv(r"C:\Users\htwal\Jupyter Projects\6a.food_bank_client_mapping\Raw Data\agency3l2f.csv")

### masterAgency cleaning

In [12]:
#take a look at each DF
masterAgency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        715 non-null    int64  
 1   Location_Id       715 non-null    int64  
 2   Location_Name     715 non-null    object 
 3   Location_Address  714 non-null    object 
 4   Location_City     711 non-null    object 
 5   Location_State    715 non-null    object 
 6   Location_Zip      709 non-null    object 
 7   Location_Status   715 non-null    object 
 8   Location_Lat      527 non-null    float64
 9   Location_Lon      527 non-null    float64
 10  nPaNo             715 non-null    object 
dtypes: float64(2), int64(2), object(7)
memory usage: 61.6+ KB


In [13]:
#pull out TEFAP or CSFP from string data, make into a series with the ID as index and keep TEFAP and CSFP to join later 
agencyServices = pd.read_csv(r"C:\Users\htwal\Jupyter Projects\6a.food_bank_client_mapping\Raw Data\AgencyService.csv")

#drop unnamed axis and set index to ID
agencyServices = agencyServices.drop('Unnamed: 0', axis=1).set_index('nPaNo')

#change to string
agencyServices['Location_Desc'] = agencyServices['Location_Desc'].astype(str)

#make a TEFAP and CSFP mask to filter the df
tefapMask = agencyServices['Location_Desc'].str.contains('TEFAP', regex=False)
csfpMask = agencyServices['Location_Desc'].str.contains('CSFP', regex=False)
agencyServices = agencyServices[tefapMask | csfpMask]

#check work and use this df to join at the end
agencyServices.value_counts()

Location_Desc
TEFAP            149
TEFAP TEMP        46
CSFP               3
dtype: int64

In [14]:
#drop unnamed column
masterAgency = masterAgency.drop(['Unnamed: 0', 'Location_Id'], axis=1)

#clean the zip codes to only keep first 5 numbers
masterAgency['Location_Zip'] = masterAgency['Location_Zip'].str[:5]

#drop all rows with missing zip codes and missing lat/lon info
masterAgency = masterAgency.dropna(subset=['Location_Zip', 'Location_Lat', 'Location_Lon'])

**You will not see the code here but**, I pulled out all the rows with null lat/lon data and saved to a separate CSV. I opened the csv in pycharm and ran it through a program to geocode all the addresses using Google and geopy. In the cell above, I dropped out all NaN rows in the lat/lon column from the masterAgency data frame, below I will load up the new .csv I made to geocode all the addresses, and add it into the masterAgency dataframe, so that all active agencies will now have associated lat/lon data.

In [15]:
#read in the csv file that was created from running the lat lon program in pycharm
latLonFix = pd.read_csv(r"C:\Users\htwal\Jupyter Projects\6a.food_bank_client_mapping\Processed Data\latlonfix.csv")

#shape latLonFix to be structured exactly the same as masterAgency
latLonFix = latLonFix[['location', 'Location_Name', 'Location_Address', 'Location_City', 'Location_State', 'Location_Zip', 'Location_Status', 'Location_Lat', 'Location_Lon', 'nPaNo']]


In [16]:
#concat the fixed dataframe to the master agency dataframe
masterAgencyFix = pd.concat([masterAgency, latLonFix], axis=0)

print(masterAgencyFix.shape)

#all the longitudes are incorrect and need to change from positive to negative values
masterAgencyFix['Location_Lon'] = masterAgencyFix['Location_Lon'].apply(lambda x: -abs(x))

#add in a master flag of 1
masterAgencyFix['Flag'] = 1

#rename nPaNo to ID
masterAgencyFix = masterAgencyFix.rename(columns={'nPaNo':'ID', 'Location_Name':'Site Name'})

#set index as 'ID'
masterAgencyFix = masterAgencyFix.set_index('ID')

#drop all columns that we dont want to get ready for joining
masterAgencyFix = masterAgencyFix.drop(['Location_Address', 'Location_City', 'Location_State', 'Location_Zip', 'Location_Status'], axis=1)

(709, 10)


In [17]:
masterAgencyFix

,Site Name,Location_Lat,Location_Lon,location,Flag
ID,,,,,
R1044,"Care and Share, Inc.",36.103999,-78.291266,NaN,1
R1048,Catholic Parish Outreach,35.808963,-78.603837,NaN,1
G1025,Christian Fellowship Homes,35.936400,-77.771900,NaN,1
R1066,Clayton Area Ministries,35.645037,-78.452015,NaN,1
D1037,Durham Exchange Club,35.975200,-78.838700,NaN,1
...,...,...,...,...,...
W1151,NoDivision Community Outreach,34.236145,-77.927859,"24 S 17th St, Wilmington, NC 28401, USA",1
W1152,Domestic Violence Shelter & Services,34.242622,-77.825292,"Wilmington, NC 28405, USA",1
W1153,NC Farmworkers’ Project,34.322059,-78.703995,"117 W Main St, Whiteville, NC 28472, USA",1


### distributionAgency cleaning

In [18]:
distributionAgency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   County                    114 non-null    object
 1   Site Name                 114 non-null    object
 2   Address                   114 non-null    object
 3   City                      114 non-null    object
 4   State                     114 non-null    object
 5   Zip                       114 non-null    int64 
 6   Contacts                  114 non-null    object
 7   Phone                     114 non-null    object
 8   Email                     112 non-null    object
 9   Distribution Information  114 non-null    object
dtypes: int64(1), object(9)
memory usage: 9.0+ KB


In [19]:
#drop columns that are not necessary
distributionAgency = distributionAgency.drop(['Contacts', 'Phone', 'Email', 'Distribution Information'], axis=1)

#need to separate the agency ID out of the name and make it the index of DF
pattern = r'([\w]+)\s-\s[\w]+'
agencyName = distributionAgency['Site Name'].str.extract(pattern, flags=re.I, expand=False).str[:5]
distributionAgency['ID'] = agencyName

#remove the ID from the Site name for comparisons when joining with other data set
patternTwo = r'[\w]+\s-\s'
distributionAgency['Site Name'] = distributionAgency['Site Name'].str.replace(patternTwo, '', regex=True)

#set index as 'ID'
distributionAgency = distributionAgency.set_index('ID')

#set zip code to type str
distributionAgency['Zip'] = distributionAgency['Zip'].astype(str)

#concat the address into a single column:
distributionAgency['concatAddress'] = (distributionAgency['Address'] + ', ' + distributionAgency['City'] + ', ' + distributionAgency['State'] + ', ' + distributionAgency['Zip'])

distributionAgency.to_csv(r"C:\Users\htwal\Jupyter Projects\6a.food_bank_client_mapping\Processed Data\distAgencyNull.csv")

In [20]:
#read in new distribution agency data fram with lat lons
distAgencyFix = pd.read_csv(r"C:\Users\htwal\Jupyter Projects\6a.food_bank_client_mapping\Processed Data\distlatlonfix.csv")

#drop unnecessary columns
distAgencyFix = distAgencyFix.drop(['Distribution', 'Unnamed: 0', 'County', 'concatAddress', 'Address', 'City', 'State', 'Zip', 'point', 'altitude'], axis=1).set_index('ID')

#add a flag showing that these agencies are being distributed to
distAgencyFix['Flag'] = 2

distAgencyFix

,Site Name,location,Location_Lat,Location_Lon,Flag
ID,,,,,
W1009,Brunswick Senior Resources,"121 Town Hall Dr, Leland, NC 28451, USA",34.238617,-78.008566,2
W1082,South Brunswick Interchurch Council,"4807 Main St, Shallotte, NC 28470, USA",33.972653,-78.385511,2
N1040,Lighthouse Community Church,"2015 US Hwy 70 E, Smithfield, NC 27577, USA",35.490888,-78.268810,2
D1020,Chatham Outreach Alliance/CORA Food Pantry,"40 Camp Dr, Pittsboro, NC 27312, USA",35.724094,-79.186309,2
W1029,Facts of Life Church Tabor City,"24678 Peacock Rd, Tabor City, NC 28463, USA",34.112227,-78.806739,2
...,...,...,...,...,...
R1512,Housing Authority of City of Goldsboro,"1009 Slaughter St, Goldsboro, NC 27530, USA",35.369130,-77.987785,2
R1512,Three in One Family Center Wayne,"700 Stevens Mill Rd, Goldsboro, NC 27530, USA",35.385530,-78.034460,2
G1246,Rhema Word Christian,"1503 Black Creek Rd SE, Wilson, NC 27893, USA",35.708174,-77.904202,2


### l2fAgency cleaning

In [21]:
l2fAgency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Status                   173 non-null    object 
 1   Organization Name        173 non-null    object 
 2   Organization Short Name  173 non-null    object 
 3   Organization Type        173 non-null    object 
 4   Address Line 1           164 non-null    object 
 5   Address Line 2           3 non-null      object 
 6   City                     173 non-null    object 
 7   State                    173 non-null    object 
 8   Zip Code                 163 non-null    float64
 9   County                   171 non-null    object 
 10  Country                  173 non-null    object 
 11  Contact 1 Name           15 non-null     object 
 12  Contact 1 Position       15 non-null     object 
 13  Contact 1 Email          6 non-null      object 
 14  Contact 1 Phone          6

In [22]:
#filter out inactive agencies
l2fAgency = l2fAgency[l2fAgency['Status'] == 'Active']

#trim down the df to only the columns we want
l2fAgency = l2fAgency[['Organization Name', 'Address Line 1', 'City', 'State', 'Zip Code', 'Programs']]

#remove the ID out of the name and create a new column
l2f_pat = r'([\w]{5})[.]*'
l2fAgency['ID'] = l2fAgency['Organization Name'].str.extract(l2f_pat, flags=re.I)

#set index as ID
l2fAgency = l2fAgency.set_index('ID')

#screen for program type and place in new column
l2fCSFP = r'(CSFP)'
l2fAgency['CSFP'] = l2fAgency['Programs'].str.extract(l2fCSFP)

l2fTEFAP = r'(TEFAP)'
l2fAgency['TEFAP'] = l2fAgency['Programs'].str.extract(l2fTEFAP)

#change zip code to string
l2fAgency['Zip Code'] = l2fAgency['Zip Code'].astype(str)

#concat the address into a single column:
l2fAgency['address'] = (l2fAgency['Address Line 1'] + ', ' + l2fAgency['City'] + ', ' + l2fAgency['State'] + ', ' + l2fAgency['Zip Code'])

l2fAgency.to_csv(r"C:\Users\htwal\Jupyter Projects\6a.food_bank_client_mapping\Processed Data\l2fAgencyNull.csv")

In [23]:
l2fAgency

,Organization Name,Address Line 1,City,State,Zip Code,Programs,CSFP,TEFAP,address
ID,,,,,,,,,
Centr,Food Bank of Central & Eastern North Carolina,1924 Capital Boulevard,Raleigh,North Carolina,27604.0,"CSFP Visit, Food Pantry Visit, TEFAP Pantry Vi...",CSFP,TEFAP,"1924 Capital Boulevard, Raleigh, North Carolin..."
Girls,Boys & Girls Clubs of the Sandhills-Logan Blake,15 Dawn Road,Pinehurst,North Carolina,28374.0,Child-School Year,NaN,NaN,"15 Dawn Road, Pinehurst, North Carolina, 28374.0"
Girls,Boys & Girls Clubs of the Sandhills-Trinity,255 South May St,Southern Pines,North Carolina,28387.0,Child-School Year,NaN,NaN,"255 South May St, Southern Pines, North Caroli..."
D1003,D1003CSFP1 - Passmore Center,103 Meadowlands Dr,Hillsborough,North Carolina,27278.0,CSFP Visit,CSFP,NaN,"103 Meadowlands Dr, Hillsborough, North Caroli..."
D1003,D1003CSFP2 - Efland Cheeks,NaN,Hillsborough,North Carolina,nan,CSFP Visit,CSFP,NaN,NaN
...,...,...,...,...,...,...,...,...,...
W1114,W1114 - YWCA Lower Cape Fear,2815 S College Rd,Wilmington,North Carolina,28409.0,CSFP Visit,CSFP,NaN,"2815 S College Rd, Wilmington, North Carolina,..."
W1133,W1133 - Cornerstone Bapstist Church,1638 Lanvale Road,Leland,North Carolina,28451.0,"Food Pantry Visit, TEFAP Pantry Visit",NaN,TEFAP,"1638 Lanvale Road, Leland, North Carolina, 284..."
W1144,W1144 - CR Resource Center,107 Mt Zion St,Lake Waccamaw,North Carolina,28450.0,CSFP Visit,CSFP,NaN,"107 Mt Zion St, Lake Waccamaw, North Carolina,..."


In [24]:
#read in the l2fAgency csv with the lat lon applied
l2fAgencyFix = pd.read_csv(r"C:\Users\htwal\Jupyter Projects\6a.food_bank_client_mapping\Processed Data\l2fLatLonFix.csv")

#drop unnecessary columns for mapping
l2fAgencyFix = l2fAgencyFix.drop(['Unnamed: 0', 'Address Line 1', 'City', 'State', 'Zip Code', 'l2f', 'address', 'point', 'altitude'], axis=1).set_index('ID')

#rename cols
l2fAgencyFix = l2fAgencyFix.rename(columns={'Organization Name':'Site Name'})

#add the flag to show the data set it came from
l2fAgencyFix['flag'] = 4

#remove the ID from the beginning of the strings
l2fAgencyFix['Site Name'] = l2fAgencyFix['Site Name'].str.replace(r'[\w]+\s-\s', '', flags=re.I, regex=True)

#add in the CSFP and TEFAP columns
l2fAgencyFix['CSFP'] = l2fAgency['CSFP']
l2fAgencyFix['TEFAP'] = l2fAgency['TEFAP']


l2fAgencyFix

,Site Name,location,Location_Lat,Location_Lon,flag,CSFP,TEFAP
ID,,,,,,,
Centr,Food Bank of Central & Eastern North Carolina,"1924 Capital Blvd, Raleigh, NC 27604, USA",35.804367,-78.615227,4,CSFP,TEFAP
Girls,Boys & Girls Clubs of the Sandhills-Logan Blake,"15 Dawn Rd, Pinehurst, NC 28374, USA",35.173393,-79.470301,4,NaN,NaN
Girls,Boys & Girls Clubs of the Sandhills-Trinity,"255 S May St, Southern Pines, NC 28387, USA",35.170139,-79.391209,4,NaN,NaN
D1003,Passmore Center,"103 Meadowlands Dr, Hillsborough, NC 27278, USA",36.063012,-79.077498,4,CSFP,NaN
D1003,Efland Cheeks,NaN,NaN,NaN,4,CSFP,NaN
...,...,...,...,...,...,...,...
W1114,YWCA Lower Cape Fear,"2815 College Rd, Wilmington, NC 28412, USA",34.194949,-77.888443,4,CSFP,NaN
W1133,Cornerstone Bapstist Church,"1638 Lanvale Rd, Leland, NC 28451, USA",34.252929,-78.056611,4,NaN,TEFAP
W1144,CR Resource Center,"107 Mt Zion St, Lake Waccamaw, NC 28450, USA",34.323056,-78.515435,4,CSFP,NaN


**Join all three agency data sets**: I will join together all three data sets that contain agency data. The purpose it to take the master dataset and join in the agencys that are currently distributed to, and join in the link2feed agencies. Then using the distribution flag we can add those columns to see if there is a total of 0, 1, or 2. 0 = active list but not on distro schedule or in l2f. 1 = active and in distro but not in l2f. 2 = active, on distro, and participating in l2f. 

In [25]:
#conduct the join
dfs = [masterAgencyFix, distAgencyFix, l2fAgencyFix, agencyServices]
agency = pd.DataFrame()
agencydf = agency.join(dfs, how='outer')

In [26]:
agencydf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 890 entries, Centr to nan
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Site Name_x     793 non-null    object 
 1   Location_Lat_x  792 non-null    float64
 2   Location_Lon_x  792 non-null    float64
 3   location_x      197 non-null    object 
 4   Flag_x          793 non-null    float64
 5   Site Name_y     209 non-null    object 
 6   location_y      209 non-null    object 
 7   Location_Lat_y  209 non-null    float64
 8   Location_Lon_y  209 non-null    float64
 9   Flag_y          209 non-null    float64
 10  Site Name       258 non-null    object 
 11  location        242 non-null    object 
 12  Location_Lat    242 non-null    float64
 13  Location_Lon    242 non-null    float64
 14  flag            258 non-null    float64
 15  CSFP            218 non-null    object 
 16  TEFAP           77 non-null     object 
 17  Location_Desc   214 non-null    obje

In [27]:
#rename the columns:
toRename = {'Site Name_x':'Site Name Master', 'Location_Lat_x': 'Lat', 'Location_Lon_x':'Lon','location_x':'Address Master', 'Flag_x': 'Flag Master', 'Site Name_y':'Site Name Dist', 'Location_Lat_y': 'Lat Dist', 'Location_Lon_y':'Lon dist','location_y':'Address dist', 'Flag_y': 'Flag Dist', 'Site Name':'Site Name l2f', 'Location_Lat': 'Lat l2f', 'Location_Lon':'Lon l2f','location':'Address l2f', 'Flag': 'Flag l2f'}
agencydf = agencydf.rename(columns=toRename)

#backfill site name master NaN values in the first column with the first agency name
def backfiller(df, col1, col2, col3):
    df[col1] = df[col1].fillna(df[col2])
    df[col1] = df[col1].fillna(df[col3])
    return df

#use the function above to backfill all the columns to have a uniform data frame where all the data is in the master columns
backfiller(agencydf, 'Site Name Master', 'Site Name Dist', 'Site Name l2f')
backfiller(agencydf, 'Lat', 'Lat Dist', 'Lat l2f')
backfiller(agencydf, 'Lon', 'Lon dist', 'Lon l2f')
backfiller(agencydf, 'Address Master', 'Address dist', 'Address l2f')

#merge the CSFP and TEFAP as necessary
agencydf['Location_Desc'] = agencydf['Location_Desc'].fillna(agencydf['TEFAP']).fillna('').str.replace('CSFP', '')
agencydf['CSFP'] = agencydf['CSFP'].fillna('')

#drop all unnecessary columns
agencydf = agencydf.drop(['TEFAP', 'Site Name Dist', 'Address dist', 'Lat Dist', 'Lon dist', 'Site Name l2f', 'Address l2f', 'Lat l2f', 'Lon l2f'], axis=1)
agencydf = agencydf.rename(columns={'Location_Desc':'TEFAP'})

#create the col which shows total points for each agency
colSum = ['Flag Master', 'Flag Dist', 'flag']
agencydf['Score'] = agencydf.loc[:,colSum].sum(axis=1)
agencydf = agencydf.drop(['Flag Master', 'Flag Dist', 'flag'], axis=1)

#finally drop out any rows where Lat or Lon is na
agencydf = agencydf.dropna(subset=['Lat', 'Lon'])

In [28]:
#the final dataset
agencydf.to_csv(r"C:\Users\htwal\Jupyter Projects\6a.food_bank_client_mapping\Final Data\agencyDfFinal.csv")
print(agencydf.shape)
agencydf

(888, 7)


,Site Name Master,Lat,Lon,Address Master,CSFP,TEFAP,Score
Centr,Food Bank of Central & Eastern North Carolina,35.804367,-78.615227,"1924 Capital Blvd, Raleigh, NC 27604, USA",CSFP,TEFAP,4.0
Centr,Food Bank of Central & Eastern North Carolina ...,35.804367,-78.615227,"1924 Capital Blvd, Raleigh, NC 27604, USA",,,4.0
D1001,ACTS of Vance County,36.325491,-78.403425,"201 S Williams St, Henderson, NC 27536, USA",,,1.0
D1002,Alliance Rehabilitative Care - Men,36.325700,-78.406500,NaN,,,1.0
D1003,Cedar Grove Community Center (Orange Co DoA),36.179094,-79.136799,"5800 NC-86, Hillsborough, NC 27278, USA",CSFP,,6.0
...,...,...,...,...,...,...,...
W1152,Domestic Violence Shelter & Services,34.242622,-77.825292,"Wilmington, NC 28405, USA",,,1.0
W1153,NC Farmworkers’ Project,34.322059,-78.703995,"117 W Main St, Whiteville, NC 28472, USA",,,1.0
W1154,St James Pentecostal Church,34.462173,-78.101154,"23585 NC-210, Currie, NC 28435, USA",CSFP,,6.0
NaN,R1553CSFP- Temple City Church,35.526740,-78.297574,"1250 S Pollock St, Selma, NC 27576, USA",,,2.0


### Conclusion to agency data cleaning: 

Overall the three data sets for locations were mostly treated the same. First I cleaned up the data sets to shape the dataframe to show the data I wanted, standardized the index with the unique IDs, filtered out NaN lat/lon data. I took all the rows in each data set that did not have lat/lon data and saved them in a csv and ran thrm through my python program that geocodes addresses using the GoogleV3 API and the Geopy library. Then, I imported those geocoded csv files and cleaned them again to make all three data sets uniform to get ready to join them into one. I also added in flags with numerical values that were totalled up in the final dataframe. The final data frame has every single ACTIVE agency, that was part of the master, distribution, or link2feed dataset, that had a valid address. Using this information, we can now apply the locations to a map, and set color coded markers to easily visualize their location and their flag data. 

Flag key: 
1. = agency present only in master agency dataset
2. = agency present only in distribution dataset
3. = agency present in master and distribution sets only
4. = agency present only in link2feed dataset
5. = agency present in master and link2feed dataset only
6. = agency present in distribution and link2feed only
7. = agency present in all three data sets. 